In [ ]:
from flask import Flask, render_template, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
import tkinter as tk
from tkinter import filedialog
from flask import Flask, render_template, request, jsonify, render_template_string
import pandas as pd


app = Flask(__name__)


model_path = 'path_to_save_model.h5'  # Replace with your model path
model = load_model(model_path)


label_encoder_path = 'label_encoder.joblib'  # Replace with your label encoder path
label_encoder = pd.read_pickle(label_encoder_path)


tkinter_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>CNN Model Deployment</title>
    <script>
        function predict() {
            var fileInput = document.getElementById("file_input");
            var file = fileInput.files[0];

            var formData = new FormData();
            formData.append('file', file);

            fetch('/predict', {
                method: 'POST',
                body: formData,
            })
            .then(response => response.json())
            .then(data => {
                alert("Predicted class: " + data.predicted_class);
            });
        }
    </script>
</head>
<body>
    <h1>CNN Model Deployment</h1>
    <input type="file" id="file_input" accept="image/*">
    <button onclick="predict()">Predict</button>
</body>
</html>
"""


@app.route('/')
def index():
    return render_template_string(tkinter_template)


@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'})

    file = request.files['file']

    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    
    temp_path = 'temp_image.jpg'
    file.save(temp_path)

    
    img = load_img(temp_path, target_size=(128, 128))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    
    prediction_probs = model.predict(img_array)[0]
    predicted_class_index = np.argmax(prediction_probs)
    predicted_class = label_encoder.classes_[predicted_class_index]

    
    os.remove(temp_path)

    return jsonify({'predicted_class': predicted_class, 'prediction_probs': prediction_probs.tolist()})


if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
